In [153]:
# create a colledge admissions algorithm

# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [154]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))


In [155]:
class Neuralstructure:
    """
    input_size: number of input nodes
    layers: number of hidden layers
    nodes_per_layer: number of nodes per hidden layer
    output_size: number of output nodes

    """
    def __init__(self, input_size: int , layers: int, nodes_per_layer: list, output_size: int):
        self.input_size = input_size
        self.layers = layers
        self.nodes_per_layer = nodes_per_layer
        self.output_size = output_size
        self.weights = []
        self.bias = []
        self.init_weights()
        self.init_bias()
        if len(nodes_per_layer) != layers:
            raise Exception("Number of layers and number of nodes per layer must be equal")
        if layers < 1:
            raise Exception("Number of layers must be greater than 0")
        if input_size < 1:
            raise Exception("Input size must be greater than 0")
        if output_size < 1:
            raise Exception("Output size must be greater than 0")
        for i in range(layers):
            if nodes_per_layer[i] < 1:
                raise Exception("Number of nodes per layer must be greater than 0")
    
    def init_weights(self):
        for i in range(self.layers):
            if i == 0:
                self.weights.append(np.random.rand(self.input_size, self.nodes_per_layer[i]))
            else:
                self.weights.append(np.random.rand(self.nodes_per_layer[i-1], self.nodes_per_layer[i]))
        self.weights.append(np.random.rand(self.nodes_per_layer[self.layers-1], self.output_size))
    
    def init_bias(self):
        for i in range(self.layers):
            self.bias.append(np.random.rand(self.nodes_per_layer[i]))
        self.bias.append(np.random.rand(self.output_size))

    

In [156]:


class NeuralNetwork:
    def __init__(self, structure: Neuralstructure , learning_rate: float):
        self.structure = structure
        self.weights = structure.weights
        self.bias = structure.bias
        self.layers = structure.layers
        self.nodes_per_layer = structure.nodes_per_layer
        self.input_size = structure.input_size
        self.output_size = structure.output_size
        self.learning_rate = learning_rate
        self.outputs = []
        self.mean = 0
        self.std = 0
        self.normalized = False

    def normalize(self, inputs: np.array):
        if not self.normalized:
            self.mean = np.mean(inputs, axis=0)
            self.std = np.std(inputs, axis=0)
            self.normalized = True
            return (inputs - self.mean) / self.std
        else:
            return (inputs - self.mean) / self.std

    def feed_forward(self, inputs: np.array):
        self.outputs = []
        for i in range(len(inputs)):
            if len(inputs[i]) != self.input_size:
                raise Exception("Input size does not match input layer size")
            self.output = []
            self.output.append(inputs[i])
            for j in range(self.layers):
                self.output.append(sigmoid(np.dot(self.output[j], self.weights[j]) + self.bias[j]))  
            self.output.append(sigmoid(np.dot(self.output[self.layers], self.weights[self.layers]) + self.bias[self.layers]))
            if self.output[-1] > 0.5:
                self.output[-1] = 1
            else:
                self.output[-1] = 0
            self.outputs.append(self.output)
        return self.outputs

    
    def backwards(self, inputs: np.array, targets: np.array):
        self.feed_forward(inputs)
        activations = self.outputs

        r = np.array([sub_list[-1] for sub_list in activations])
        mse = np.mean(np.square(targets - r))

        for i in range(len(activations)):
            
            a = activations[i]
            output_error = targets[i] - a[-1]

            for j in range(self.layers, 0, -1):
                error = output_error * sigmoid_prime(a[j])
                output_error = error.dot(self.weights[j].T[0])

                self.weights[j] += self.learning_rate * a[j].T.dot(error)
                self.bias[j] += self.learning_rate * np.sum(error, axis=0, keepdims=True)

            error = output_error * sigmoid_prime(a[0])

            
            self.weights[0] += self.learning_rate * inputs[i].T.dot(error)
            self.bias[0] += self.learning_rate * np.sum(error, axis=0, keepdims=True)
        return mse 


    
    def train(self, inputs: np.array, targets: np.array, epochs: int):
        """
        inputs: array of inputs
        targets: array of targets
        """
        for i in range(epochs):
            mse = self.backwards(inputs, targets)
        if epochs % 100 == 0:
            print(mse)


           
    

In [157]:
def node_structure(input_size: int, layers:int):
    nodes_per_layer = []
    average = input_size / layers

    for i in range(layers):
        n = int(round(input_size - (average * i))) + 1
        if n < 1:
            nodes_per_layer.append(1)
        else:
            nodes_per_layer.append(n)
    return nodes_per_layer

In [158]:
def setup(layers, output_size):
    """ 
    setup the neural network
    layers: number of hidden layers
    output_size: number of output nodes
    filename:'input.csv'
    """
    data = pd.read_csv('input.csv')
    header = data.columns.to_list()
    result = header[-1]
    y_train = data[result]
    x_train = data.drop(result, axis=1)
    x_train = x_train.fillna(0)
    y_train = y_train.values
    x_train = x_train.values
    input_size = len(header) - 1
    nodes = node_structure(input_size, layers)

    ns = Neuralstructure(input_size, layers, nodes, output_size)
    nn = NeuralNetwork(ns, 0.1)
    x_train = nn.normalize(x_train)
    nn.train(x_train, y_train, 1500)
    return nn

In [163]:
network = setup(3, 1)



0.5


In [164]:
network.nodes_per_layer

[2, 1, 1]

In [161]:
def predict(network, inputs):
    network.normalize(inputs)
    preditions = network.feed_forward(inputs)
    predictions = [sub_list[-1] for sub_list in preditions]
    
    return predictions


In [162]:
input = np.array([[1,1], [1,0], [0,1], [0,0]])
print(predict(network, input))

[0, 0, 0, 0]
